## Loading Packages

In [1]:
!pip install -q transformers
!pip install -q torchinfo
!pip install -U -q datasets fsspec huggingface_hub
!pip install -q evaluate

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import transformers
import evaluate

from datasets import load_dataset, concatenate_datasets
from torchinfo import summary

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoConfig
from transformers import TrainingArguments, Trainer, pipeline

from typing import Dict, Any
import copy
from pathlib import Path
import json
import os

## Dataset preparation

In [3]:
imdb_dataset = load_dataset("imdb")

train_tiny = imdb_dataset['train'].shuffle().select(range(100))
train_small = imdb_dataset['train'].shuffle().select(range(1000))
train_medium = imdb_dataset['train'].shuffle().select(range(2500))
train_large = imdb_dataset['train'].shuffle().select(range(10000))


test_tiny = imdb_dataset['test'].shuffle().select(range(100))
test_small = imdb_dataset['test'].shuffle().select(range(1000))
test_medium = imdb_dataset['test'].shuffle().select(range(5000))
test_large = imdb_dataset['test'].shuffle().select(range(10000))

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
MAX_SEQUENCE_LENGTH = 100

def preprocess_imdb(data, tokenizer):
    review_text = data['text']

    encoded = tokenizer.batch_encode_plus(
            review_text,
            max_length=MAX_SEQUENCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )

    return encoded

In [5]:
rte_dataset = load_dataset("glue", "rte")

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [6]:
qqp_dataset = load_dataset("glue", "qqp")

qqp_train = qqp_dataset["train"].shuffle().select(range(2500))
qqp_test = qqp_dataset["validation"].shuffle().select(range(4000))

train-00000-of-00001.parquet:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.73M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [7]:
qnli_dataset = load_dataset("glue", "qnli")
qnli_train = qnli_dataset["train"].shuffle().select(range(2500))
qnli_test = qnli_dataset["validation"].shuffle().select(range(4000))

train-00000-of-00001.parquet:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/872k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/877k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

In [8]:
mnli_dataset = load_dataset("glue", "mnli")
validation_all = concatenate_datasets([
    mnli_dataset["validation_matched"],
    mnli_dataset["validation_mismatched"]
])

mnli_train = mnli_dataset["train"].shuffle().select(range(2500))
mnli_test = validation_all.shuffle().select(range(4000))

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

test_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [9]:
wnli_dataset = load_dataset("glue", "wnli")

train-00000-of-00001.parquet:   0%|          | 0.00/38.8k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/13.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/635 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/71 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/146 [00:00<?, ? examples/s]

In [10]:
def preprocess_rte(examples, tokenizer):
    return tokenizer(
        text=examples["sentence1"],
        text_pair=examples["sentence2"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors='pt' #important, don't get cuda error.
    )

def preprocess_qqp(examples, tokenizer):
    return tokenizer(
        text=examples["question1"],
        text_pair=examples["question2"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors='pt'
    )

def preprocess_qnli(examples, tokenizer):
    return tokenizer(
        text=examples["question"],
        text_pair=examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors='pt'
    )

def preprocess_mnli(examples, tokenizer):
    return tokenizer(
        text=examples["premise"],
        text_pair=examples["hypothesis"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors='pt'
    )

def preprocess_wnli(examples, tokenizer):
    return tokenizer(
        text=examples["sentence1"],
        text_pair=examples["sentence2"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors='pt'
    )


## Loading Functions

In [11]:
metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
def fine_tune_classification_model(model, tokenizer, train_data, dev_data,
                                   num_epochs=1, output_dir="temp",
                                   batch_size=16,
                                   data_preprocessor=preprocess_imdb,
                                   save_path=None, resume_from=None):
    """
    Fine tune model on given dataset.
    """
    # Preprocess data
    train_data_processed = train_data.map(data_preprocessor, batched=True,
                                         fn_kwargs={'tokenizer': tokenizer})
    dev_data_processed = dev_data.map(data_preprocessor, batched=True,
                                     fn_kwargs={'tokenizer': tokenizer})

    # Training configuration
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        save_on_each_node=True,
        report_to='none',
        load_best_model_at_end=False,
    )

    # Create trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_processed,
        eval_dataset=dev_data_processed,
        compute_metrics=compute_metrics
    )

    # Train with optional resume
    if resume_from:
        trainer.train(resume_from_checkpoint=resume_from)
    else:
        trainer.train()

    # Save final model state
    if save_path:
        # Save FULL state instead of just model
        trainer.save_model(save_path)

    return trainer

In [13]:
def remove_svs_decile(linear_layer, decile_to_remove):
    """
    Remove a specific decile of singular values from a linear layer.
    :param linear_layer: nn.Linear module
    :param decile_to_remove: int in [0, 9], where 0 = top 10%, 9 = bottom 10%
    """
    with torch.no_grad():
        W = linear_layer.weight.data
        U, s, Vh = torch.linalg.svd(W.float(), full_matrices=False)

        n = len(s)
        decile_size = n // 10
        start = decile_to_remove * decile_size
        end = (decile_to_remove + 1) * decile_size if decile_to_remove < 9 else n

        s_trunc = s.clone()
        s_trunc[start:end] = 0

        W_trunc = U @ torch.diag(s_trunc) @ Vh
        linear_layer.weight.data = W_trunc.to(W.dtype)



def remove_svs_from_bert_decile(
                                model,
                                decile_to_remove,
                                layer_type="all",
                                layers="all",  #specify layers to target.
                            ):
    """
    Remove a decile of SVs from specific layers/matrices.

    :param layers: "all", list of indices (e.g., [0,1,2]), or "late" (last 4)
    :param layer_type: "all", "query", "key", "value", "output", "ffn"
    """
    if layers == "late":
        layers = list(range(len(model.bert.encoder.layer) - 4, len(model.bert.encoder.layer)))
    elif layers == "all":
        layers = range(len(model.bert.encoder.layer))

    for layer_idx in layers:
        layer = model.bert.encoder.layer[layer_idx]
        if layer_type in ["all", "query"]:
            remove_svs_decile(layer.attention.self.query, decile_to_remove)
        if layer_type in ["all", "key"]:
            remove_svs_decile(layer.attention.self.key, decile_to_remove)
        if layer_type in ["all", "value"]:
            remove_svs_decile(layer.attention.self.value, decile_to_remove)
        if layer_type in ["all", "output"]:
            remove_svs_decile(layer.attention.output.dense, decile_to_remove)
        if layer_type in ["all", "ffn"]:
            remove_svs_decile(layer.intermediate.dense, decile_to_remove)
            remove_svs_decile(layer.output.dense, decile_to_remove)

def get_minimal_eval_args(model):
    """Create minimal TrainingArguments for evaluation"""
    return TrainingArguments(
        output_dir="./temp_eval",  # Temporary directory
        per_device_eval_batch_size=16,
        report_to='none',  # Disable logging
        no_cuda=False if torch.cuda.is_available() else True,
        fp16=model.config.torch_dtype == torch.float16,
    )

def evaluate_decile_removal(
        control_model,
        tokenizer,
        compute_metrics,
        test_data,
        layer_type="all",
        layers="all"
    ):
    """
    Evaluate validation accuracy after removing each decile of SVs.
    Returns a list of accuracies, one per decile.
    """
    trainer_args = TrainingArguments(
                                      output_dir="./tmp_eval",      # required, but can be temp
                                      per_device_eval_batch_size=32,  # or whatever you used
                                      do_train=False,
                                      do_eval=True,
                                      logging_strategy="no",  # suppress extra logs
                                      save_strategy="no",
                                      report_to="none"
                                      )
    accuracies = []

    for decile in range(10):
        print(f"\n=== Evaluating decile {decile} removal ===")
        # Clone model in memory instead of reloading from disk
        treatment_model = copy.deepcopy(control_model)

        # Apply decile-specific SV removal
        remove_svs_from_bert_decile(
            treatment_model,
            decile_to_remove=decile,
            layer_type=layer_type,
            layers=layers
        )

        treatment_trainer = Trainer(
            model=treatment_model,
            args=trainer_args,
            compute_metrics=compute_metrics,
            eval_dataset=test_data
        )
        results = treatment_trainer.evaluate()
        accuracies.append(results["eval_accuracy"])

    return accuracies

## Training Models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#loading base stuff.
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_model = AutoModel.from_pretrained('bert-base-cased')

#assign model name to variable for convenience
model_checkpoint_name = "bert-base-cased"

#initialize tokenizer using model checkpoint name
bert_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

### IMDb (easy)

In [ ]:
test_processed = test_large.map(preprocess_imdb, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})

Initial training (1 epoch)

In [ ]:
model_imdb = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
trainer_imdb = fine_tune_classification_model(
    model_imdb,
    bert_tokenizer,
    train_large,
    test_large,
    num_epochs=1,
    output_dir="temp_imdb_1",
    save_path="drive/MyDrive/Summer2025/w266/Project/models/imdb_epoch1"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.444800,0.345577,0.848100


Continue training (1 -> 2 epochs)

In [ ]:
#find latest checkpoint
checkpoint_dir = max(
    [f for f in Path("temp_imdb_1").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_epoch1 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/imdb_epoch1"
)

# Train additional epoch
trainer2 = fine_tune_classification_model(
    model_epoch1,
    bert_tokenizer,
    train_large,
    test_large,
    num_epochs=2, #resume
    output_dir="temp_imdb_2",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/imdb_epoch2",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
2,0.266600,0.391251,0.854200


Continue training (2 --> 3 epochs)

In [ ]:
#find latest checkpoint
checkpoint_dir = max(
    [f for f in Path("temp_imdb_2").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_epoch2 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/imdb_epoch2"
)

# Train additional epoch
trainer3 = fine_tune_classification_model(
    model_epoch2,
    bert_tokenizer,
    train_large,
    test_large,
    num_epochs=3,
    output_dir="temp_imdb_3",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/imdb_epoch3",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
3,0.170700,0.497933,0.859000


In [ ]:
#find latest checkpoint
checkpoint_dir = max(
    [f for f in Path("temp_imdb_3").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_epoch3 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/imdb_epoch3"
)

trainer10 = fine_tune_classification_model(
    model_epoch3,
    bert_tokenizer,
    train_large,
    test_large,
    num_epochs=10, # already trained 3 epochs --> 3 + 7 = 10 total.
    output_dir="temp_imdb_10",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/imdb_epoch10",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
4,0.203600,0.457572,0.840200
5,0.121100,0.679779,0.834700
6,0.088600,0.815664,0.842700
7,0.051800,1.015398,0.845800
8,0.017800,1.155892,0.848200
9,0.005300,1.273104,0.847500
10,0.002600,1.308429,0.848300


### RTE (medium)

In [ ]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Initial training (1 epoch)

In [ ]:
model_rte = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
trainer_rte = fine_tune_classification_model(
    model_rte,
    bert_tokenizer,
    train_data=rte_dataset["train"],
    dev_data=rte_dataset["validation"],
    data_preprocessor=preprocess_rte,
    num_epochs=1,
    output_dir="temp_rte_1",
    save_path="drive/MyDrive/Summer2025/w266/Project/models/rte_epoch1"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.656533,0.631769


Continue training (1 --> 2 epochs)

In [ ]:
#find latest checkpoint
checkpoint_dir = max(
    [f for f in Path("temp_rte_1").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_rte1 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/rte_epoch1"
)

# Train additional epoch
trainer_rte2 = fine_tune_classification_model(
    model_rte1,
    bert_tokenizer,
    train_data=rte_dataset["train"],
    dev_data=rte_dataset["validation"],
    data_preprocessor=preprocess_rte,
    num_epochs=2, #resume
    output_dir="temp_rte_2",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/rte_epoch2",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
2,No log,0.626228,0.657040


Continue training (2 --> 3 epochs)

In [ ]:
#find latest checkpoint
checkpoint_dir = max(
    [f for f in Path("temp_rte_2").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_rte2 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/rte_epoch2"
)

trainer_rte3 = fine_tune_classification_model(
    model_rte2,
    bert_tokenizer,
    train_data=rte_dataset["train"],
    dev_data=rte_dataset["validation"],
    data_preprocessor=preprocess_rte,
    num_epochs=3, #resume
    output_dir="temp_rte_3",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/rte_epoch3",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
3,No log,0.772841,0.664260


Continue training (3 --> 10 epochs)

In [ ]:
#find latest checkpoint
checkpoint_dir = max(
    [f for f in Path("temp_rte_3").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_rte3 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/rte_epoch3"
)

# Train additional epoch
trainer_rte10 = fine_tune_classification_model(
    model_rte3,
    bert_tokenizer,
    train_data=rte_dataset["train"],
    dev_data=rte_dataset["validation"],
    data_preprocessor=preprocess_rte,
    num_epochs=10, #resume
    output_dir="temp_rte_10",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/rte_epoch10",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
4,0.354600,0.881907,0.628159
5,0.354600,1.273966,0.657040
6,0.354600,1.446766,0.667870
7,0.219600,2.033015,0.635379
8,0.219600,2.485233,0.628159
9,0.219600,2.550749,0.638989
10,0.046900,2.556892,0.642599


### QQP (medium)

In [ ]:
#we downsample for time reasons.
model_qqp = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
trainer_qqp = fine_tune_classification_model(
    model_qqp,
    bert_tokenizer,
    train_data=qqp_train,
    dev_data=qqp_test,
    data_preprocessor=preprocess_qqp,
    num_epochs=1,
    batch_size=32,
    output_dir="temp_qqp_1",
    save_path="drive/MyDrive/Summer2025/w266/Project/models/qqp_epoch1"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.457042,0.770250


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_qqp_1").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_qqp1 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/qqp_epoch1"
)

# Train additional epoch
trainer_qqp2 = fine_tune_classification_model(
    model_qqp1,
    bert_tokenizer,
    train_data=qqp_train,
    dev_data=qqp_test,
    data_preprocessor=preprocess_qqp,
    num_epochs=2, #resume,
    batch_size=32,
    output_dir="temp_qqp_2",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/qqp_epoch2",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
2,No log,0.449391,0.793250


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_qqp_2").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_qqp2 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/qqp_epoch2"
)

# Train additional epoch
trainer_qqp3 = fine_tune_classification_model(
    model_qqp2,
    bert_tokenizer,
    train_data=qqp_train,
    dev_data=qqp_test,
    data_preprocessor=preprocess_qqp,
    num_epochs=3, #resume,
    batch_size=32,
    output_dir="temp_qqp_3",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/qqp_epoch3",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
3,No log,0.495800,0.794750


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_qqp_3").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_qqp3 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/qqp_epoch3"
)

# Train additional epoch
trainer_qqp10 = fine_tune_classification_model(
    model_qqp3,
    bert_tokenizer,
    train_data=qqp_train,
    dev_data=qqp_test,
    data_preprocessor=preprocess_qqp,
    num_epochs=10, #resume,
    batch_size=32,
    output_dir="temp_qqp_10",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/qqp_epoch10",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
4,No log,0.623229,0.769750
5,No log,0.727512,0.788250
6,No log,0.875514,0.793500
7,0.123400,0.953322,0.783250
8,0.123400,0.991377,0.783000
9,0.123400,1.009651,0.793250
10,0.123400,1.003528,0.789250


### QNLI (hard)

In [ ]:
#we downsample for time reasons.
model_qnli = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
trainer_qnli = fine_tune_classification_model(
    model_qnli,
    bert_tokenizer,
    train_data=qnli_train,
    dev_data=qnli_test,
    data_preprocessor=preprocess_qnli,
    num_epochs=1,
    batch_size=32,
    output_dir="temp_qnli_1",
    save_path="drive/MyDrive/Summer2025/w266/Project/models/qnli_epoch1"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.406567,0.824000


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_qnli_1").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_qnli1 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/qnli_epoch1"
)

# Train additional epoch
trainer_qnli2 = fine_tune_classification_model(
    model_qnli1,
    bert_tokenizer,
    train_data=qnli_train,
    dev_data=qnli_test,
    data_preprocessor=preprocess_qnli,
    num_epochs=2, #resume,
    batch_size=32,
    output_dir="temp_qnli_2",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/qnli_epoch2",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
2,No log,0.411307,0.824750


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_qnli_2").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_qnli2 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/qnli_epoch2"
)

# Train additional epoch
trainer_qnli3 = fine_tune_classification_model(
    model_qnli2,
    bert_tokenizer,
    train_data=qnli_train,
    dev_data=qnli_test,
    data_preprocessor=preprocess_qnli,
    num_epochs=3, #resume,
    batch_size=32,
    output_dir="temp_qnli_3",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/qnli_epoch3",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
3,No log,0.479552,0.821750


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_qnli_3").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_qnli3 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/qnli_epoch3"
)

# Train additional epoch
trainer_qnli10 = fine_tune_classification_model(
    model_qnli3,
    bert_tokenizer,
    train_data=qnli_train,
    dev_data=qnli_test,
    data_preprocessor=preprocess_qnli,
    num_epochs=10, #resume,
    batch_size=32,
    output_dir="temp_qnli_10",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/qnli_epoch10",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
4,No log,0.622264,0.819000
5,No log,0.747037,0.813750
6,No log,0.791892,0.817500
7,0.089400,0.760011,0.828250
8,0.089400,0.845041,0.822250
9,0.089400,0.865367,0.829000
10,0.089400,0.867959,0.828250


### MNLI (hard)

In [ ]:
#we downsample for time reasons.
model_mnli = AutoModelForSequenceClassification.from_pretrained('bert-base-cased',
                                                                num_labels=3)
trainer_mnli = fine_tune_classification_model(
    model_mnli,
    bert_tokenizer,
    train_data=mnli_train,
    dev_data=mnli_test,
    data_preprocessor=preprocess_mnli,
    num_epochs=1,
    batch_size=32,
    output_dir="temp_mnli_1",
    save_path="drive/MyDrive/Summer2025/w266/Project/models/mnli_epoch1"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.895806,0.611000


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_mnli_1").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_mnli1 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/mnli_epoch1"
)

# Train additional epoch
trainer_mnli2 = fine_tune_classification_model(
    model_mnli1,
    bert_tokenizer,
    train_data=mnli_train,
    dev_data=mnli_test,
    data_preprocessor=preprocess_mnli,
    num_epochs=2, #resume,
    batch_size=32,
    output_dir="temp_mnli_2",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/mnli_epoch2",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
2,No log,0.821615,0.644000


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_mnli_2").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_mnli2 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/mnli_epoch2"
)

# Train additional epoch
trainer_mnli3 = fine_tune_classification_model(
    model_mnli2,
    bert_tokenizer,
    train_data=mnli_train,
    dev_data=mnli_test,
    data_preprocessor=preprocess_mnli,
    num_epochs=3, #resume,
    batch_size=32,
    output_dir="temp_mnli_3",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/mnli_epoch3",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
3,No log,0.820207,0.661250


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_mnli_3").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_mnli3 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/mnli_epoch3"
)

# Train additional epoch
trainer_mnli10 = fine_tune_classification_model(
    model_mnli3,
    bert_tokenizer,
    train_data=mnli_train,
    dev_data=mnli_test,
    data_preprocessor=preprocess_mnli,
    num_epochs=10, #resume,
    batch_size=32,
    output_dir="temp_mnli_10",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/mnli_epoch10",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
4,No log,0.884828,0.666750
5,No log,0.953281,0.684750
6,No log,1.072345,0.680750
7,0.291400,1.257914,0.677750
8,0.291400,1.370241,0.677000
9,0.291400,1.453277,0.680500
10,0.291400,1.443409,0.681250


### WNLI (very hard, be careful because there might be some bugs with there being devset  == test set in early versions??) also smaller gradient updates

In [ ]:
model_wnli = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
wnli_train = wnli_dataset["train"]
wnli_test  = wnli_dataset["validation"]


trainer_wnli = fine_tune_classification_model(
    model_wnli,
    bert_tokenizer,
    train_data=wnli_train,
    dev_data=wnli_test,
    data_preprocessor=preprocess_wnli,
    num_epochs=1,
    batch_size=8,
    output_dir="temp_wnli_1",
    save_path="drive/MyDrive/Summer2025/w266/Project/models/wnli_epoch1"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.686209,0.563380


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_wnli_1").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_wnli1 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/wnli_epoch1"
)

# Train additional epoch
trainer_wnli2 = fine_tune_classification_model(
    model_wnli1,
    bert_tokenizer,
    train_data=wnli_train,
    dev_data=wnli_test,
    data_preprocessor=preprocess_wnli,
    num_epochs=2, #resume,
    batch_size=8,
    output_dir="temp_wnli_2",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/wnli_epoch2",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
2,No log,0.691785,0.563380


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_wnli_2").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_wnli2 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/wnli_epoch2"
)

# Train additional epoch
trainer_wnli3 = fine_tune_classification_model(
    model_wnli2,
    bert_tokenizer,
    train_data=wnli_train,
    dev_data=wnli_test,
    data_preprocessor=preprocess_wnli,
    num_epochs=3, #resume,
    batch_size=8,
    output_dir="temp_wnli_3",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/wnli_epoch3",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
3,No log,0.695584,0.352113


In [ ]:
checkpoint_dir = max(
    [f for f in Path("temp_wnli_3").iterdir() if f.is_dir() and "checkpoint" in f.name],
    key=os.path.getmtime
)

#load initial model
model_wnli3 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/wnli_epoch3"
)

# Train additional epoch
trainer_wnli10 = fine_tune_classification_model(
    model_wnli3,
    bert_tokenizer,
    train_data=wnli_train,
    dev_data=wnli_test,
    data_preprocessor=preprocess_wnli,
    num_epochs=10, #resume,
    batch_size=8,
    output_dir="temp_wnli_10",  # New temp dir
    save_path="drive/MyDrive/Summer2025/w266/Project/models/wnli_epoch10",
    resume_from=str(checkpoint_dir)  # Resume from checkpoint
)

Epoch,Training Loss,Validation Loss,Accuracy
4,No log,0.712666,0.436620
5,No log,0.699393,0.436620
6,No log,0.709221,0.436620
7,0.702300,0.690531,0.563380
8,0.702300,0.709695,0.436620
9,0.702300,0.704419,0.338028
10,0.702300,0.711611,0.281690


## SV Experiments


### Functions & Drive Mount


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
def evaluate_model(model, test_data):
    """Compute accuracy of unmodified model"""
    trainer_args = TrainingArguments(
                                      output_dir="./tmp_eval",      # required, but can be temp
                                      per_device_eval_batch_size=32,  # or whatever you used
                                      do_train=False,
                                      do_eval=True,
                                      logging_strategy="no",  # suppress extra logs
                                      save_strategy="no",
                                      report_to="none"
                                      )

    treatment_trainer = Trainer(
                model=model,
                args=trainer_args,
                compute_metrics=compute_metrics,
                eval_dataset=test_data
            )

    res = treatment_trainer.evaluate()
    return res["eval_accuracy"]

In [16]:
def run_experiment(dataset_name: str,
                   test_data,
                   tokenizer):
    EPOCHS = [1, 2, 3, 10]
    LAYER_TYPES = ["all", "query", "key", "value", "output", "ffn"]
    BASE_PATH = "drive/MyDrive/Summer2025/w266/Project"

    results = {}

    print(f"\n=== STARTING {dataset_name.upper()} EXPERIMENTS ===")
    for epoch in EPOCHS:
        epoch_key = f"epoch{epoch}"
        results[epoch_key] = {}
        print(f"\n=== EPOCH {epoch} ===")
        try:
            # 1. Load model and tokenizer
            model_path = f"{BASE_PATH}/models/{dataset_name}_{epoch_key}"
            model = AutoModelForSequenceClassification.from_pretrained(model_path)

            # 2. Evaluate control model (no ablation)
            control_acc = evaluate_model(model, test_data)
            results[epoch_key]["control"] = control_acc
            print(f"  {epoch_key} | Control: {control_acc:.4f}")

            # 3. Run ablation for each layer type
            ablation_results = {}
            for layer_type in LAYER_TYPES:
                accuracies = evaluate_decile_removal(
                    control_model=model,
                    tokenizer=tokenizer,
                    test_data=test_data,
                    compute_metrics=compute_metrics,
                    layer_type=layer_type
                )

                ablation_results[layer_type] = accuracies
                print(f"    {layer_type}: {accuracies}")

            results[epoch_key]["ablation"] = ablation_results

            # Free GPU memory
            del model
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"ERROR: {dataset_name}_{epoch_key} failed - {str(e)}")
            results[epoch_key]["error"] = str(e)

    # 4. Save results (moved outside of the loop)
    output_path = f"{BASE_PATH}/results/{dataset_name}_results.json"
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)

    print(f"=== SAVED RESULTS TO {output_path} ===")
    return results

In [17]:
def run_ffn_experiment(dataset_name: str,
                   test_data,
                   tokenizer):
    EPOCHS = [1, 2, 3, 10]
    LAYER_GROUPS = {
        "0-3": [0, 1, 2, 3],
        "4-7": [4, 5, 6, 7],
        "8-11": [8, 9, 10, 11]
    }
    BASE_PATH = "drive/MyDrive/Summer2025/w266/Project"

    results = {}

    print(f"\n=== STARTING {dataset_name.upper()} EXPERIMENTS ===")
    for epoch in EPOCHS:
        epoch_key = f"epoch{epoch}"
        results[epoch_key] = {}
        print(f"\n=== EPOCH {epoch} ===")
        try:
            # 1. Load model and tokenizer
            model_path = f"{BASE_PATH}/models/{dataset_name}_{epoch_key}"
            model = AutoModelForSequenceClassification.from_pretrained(model_path)

            # 2. Evaluate control model (no ablation)
            control_acc = evaluate_model(model, test_data)
            results[epoch_key]["control"] = control_acc
            print(f"  {epoch_key} | Control: {control_acc:.4f}")

            # 3. Run ablation for each layer group
            ablation_results = {}
            for group_name, layer_indices in LAYER_GROUPS.items():
                accuracies = evaluate_decile_removal(
                    control_model=model,
                    tokenizer=tokenizer,
                    test_data=test_data,
                    compute_metrics=compute_metrics,
                    layer_type="ffn",  # Fixed to FFN layers
                    layers=layer_indices  # Use current layer group
                )

                ablation_results[group_name] = accuracies
                print(f"    Group {group_name}: {accuracies}")

            results[epoch_key]["ablation"] = ablation_results

            # Free GPU memory
            del model
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"ERROR: {dataset_name}_{epoch_key} failed - {str(e)}")
            results[epoch_key]["error"] = str(e)

    # 4. Save results with new filename
    output_path = f"{BASE_PATH}/results/{dataset_name}_layer_results.json"
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)

    print(f"=== SAVED RESULTS TO {output_path} ===")
    return results

### Ablation Experiments

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

below took 49 minutes


In [ ]:
imdb_test = test_medium.map(preprocess_imdb, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
imdb_res = run_experiment(dataset_name="imdb",
                          test_data=imdb_test,
                          tokenizer=bert_tokenizer)

below took 12 minutes


In [ ]:
rte_test = rte_dataset["validation"].map(preprocess_rte, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
rte_res = run_experiment(dataset_name="rte",
                          test_data=rte_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/277 [00:00<?, ? examples/s]


=== STARTING RTE EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.6318

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    ffn: [0.4729241877256318, 0.6498194945848376, 0.6606498194945848, 0.6389891696750902, 0.6389891696750902, 0.6498194945848376, 0.6353790613718412, 0.6534296028880866, 0.6353790613718412, 0.6462093862815884]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/rte_results.json ===


below took 2 hours.

In [ ]:
#qqp
qqp_test = qqp_test.map(preprocess_qqp, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
qqp_res = run_experiment(dataset_name="qqp",
                          test_data=qqp_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]


=== STARTING QQP EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.7600

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    ffn: [0.36375, 0.73225, 0.7535, 0.78075, 0.77675, 0.7735, 0.78075, 0.77575, 0.7825, 0.77475]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/qqp_results.json ===


below took 2 hours

In [ ]:
#qnli
qnli_test = qnli_test.map(preprocess_qnli, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
qnli_res = run_experiment(dataset_name="qnli",
                          test_data=qnli_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]


=== STARTING QNLI EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.8193

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    ffn: [0.494, 0.63375, 0.7525, 0.78425, 0.80475, 0.812, 0.81375, 0.81875, 0.82225, 0.82725]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/qnli_results.json ===


In [ ]:
#mnli
mnli_test = mnli_test.map(preprocess_mnli, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
mnli_res = run_experiment(dataset_name="mnli",
                          test_data=mnli_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]


=== STARTING MNLI EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.6170

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    ffn: [0.36, 0.5265, 0.651, 0.66775, 0.6835, 0.69075, 0.69675, 0.69725, 0.68925, 0.67175]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/mnli_results.json ===


In [ ]:
model_rte1 = AutoModelForSequenceClassification.from_pretrained(
    "drive/MyDrive/Summer2025/w266/Project/models/rte_epoch1"
)

In [ ]:
test_processed = rte_dataset['validation'].map(preprocess_rte, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})

accuracies = evaluate_decile_removal(
    control_model=model_rte1,
    tokenizer=bert_tokenizer,
    test_data=test_processed,
    compute_metrics=compute_metrics,
    layer_type="query", #all, query, key, value, output, ffn
    layers=[0, 1, 2, 3]
)



=== Evaluating decile 0 removal ===



=== Evaluating decile 1 removal ===



=== Evaluating decile 2 removal ===



=== Evaluating decile 3 removal ===



=== Evaluating decile 4 removal ===



=== Evaluating decile 5 removal ===



=== Evaluating decile 6 removal ===



=== Evaluating decile 7 removal ===



=== Evaluating decile 8 removal ===



=== Evaluating decile 9 removal ===


In [ ]:
res = {}
for layer_type in ["query", "key", "value", "output", "ffn"]:
  accs = evaluate_decile_removal(
    control_model=trainer3.model,
    tokenizer=tokenizer,
    trainer_args=trainer3.args,
    compute_metrics=trainer3.compute_metrics,
    test_data=test_processed,
    layer_type=layer_type, #all, query, key, value, output, ffn
    layers="all"
)
  res[layer_type] = accs


=== Evaluating decile 0 removal ===



=== Evaluating decile 1 removal ===



=== Evaluating decile 2 removal ===



=== Evaluating decile 3 removal ===



=== Evaluating decile 4 removal ===


...


=== Evaluating decile 5 removal ===



=== Evaluating decile 6 removal ===



=== Evaluating decile 7 removal ===



=== Evaluating decile 8 removal ===



=== Evaluating decile 9 removal ===


### FFN Layer Experiments

In [18]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [19]:
imdb_test = test_medium.map(preprocess_imdb, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
imdb_res = run_ffn_experiment(dataset_name="imdb",
                          test_data=imdb_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]


=== STARTING IMDB EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.8474

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    Group 8-11: [0.8214, 0.8432, 0.8442, 0.8442, 0.8442, 0.8432, 0.8442, 0.8436, 0.8454, 0.8462]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/imdb_layer_results.json ===


In [20]:
rte_test = rte_dataset["validation"].map(preprocess_rte, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
rte_res = run_ffn_experiment(dataset_name="rte",
                          test_data=rte_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/277 [00:00<?, ? examples/s]


=== STARTING RTE EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.6318

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    Group 8-11: [0.4981949458483754, 0.6570397111913358, 0.6534296028880866, 0.6570397111913358, 0.6534296028880866, 0.6570397111913358, 0.6534296028880866, 0.6534296028880866, 0.6498194945848376, 0.631768953068592]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/rte_layer_results.json ===


In [21]:
#qqp
qqp_test = qqp_test.map(preprocess_qqp, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
qqp_res = run_ffn_experiment(dataset_name="qqp",
                          test_data=qqp_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]


=== STARTING QQP EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.7600

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    Group 8-11: [0.36825, 0.774, 0.766, 0.77425, 0.77725, 0.77225, 0.77325, 0.78125, 0.78325, 0.7825]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/qqp_layer_results.json ===


In [22]:
#qnli
qnli_test = qnli_test.map(preprocess_qnli, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
qnli_res = run_ffn_experiment(dataset_name="qnli",
                          test_data=qnli_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]


=== STARTING QNLI EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.8233

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    Group 8-11: [0.49925, 0.8275, 0.829, 0.8255, 0.83, 0.83, 0.829, 0.838, 0.8345, 0.8395]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/qnli_layer_results.json ===


In [23]:
#mnli
mnli_test = mnli_test.map(preprocess_mnli, batched=True, fn_kwargs={'tokenizer': bert_tokenizer})
mnli_res = run_ffn_experiment(dataset_name="mnli",
                          test_data=mnli_test,
                          tokenizer=bert_tokenizer)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]


=== STARTING MNLI EXPERIMENTS ===

=== EPOCH 1 ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


  epoch1 | Control: 0.6035

=== Evaluating decile 0 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...


=== Evaluating decile 8 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluating decile 9 removal ===


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


    Group 8-11: [0.4195, 0.66825, 0.66975, 0.679, 0.6785, 0.68375, 0.67475, 0.679, 0.6755, 0.67075]
=== SAVED RESULTS TO drive/MyDrive/Summer2025/w266/Project/results/mnli_layer_results.json ===
